# Reconstruction Evaluation
In this notebook, we will reconstruct images with different models, and evaluate the reconstruction performance of different models.
## Helper Functions

In [1]:
import cv2
import os
import re
import glob
import numpy as np
import torch
import dnnlib
import legacy
from metrics import metric_utils
from torch.nn.functional import cosine_similarity
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import random


def resize_crop(img_dir, resize_by=1., resolution=512, brightness_norm=True, brightness_mean=107.2, locations=None):
    if locations is None:
        locations = ["left", "right"]
    img = cv2.imread(img_dir, cv2.IMREAD_UNCHANGED)
    if img.dtype != np.uint8:
        img = np.uint8(img / 256)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_shape = img.shape
    resize_shape = np.array([img_shape[1] * resize_by, img_shape[0] * resize_by], dtype=int)
    if resize_by != 1:
        img = cv2.resize(img, resize_shape, cv2.INTER_LANCZOS4)
    imgs = []
    for location in locations:
        if location == "left":
            new_img = img[(resize_shape[1] - resolution) // 2:(resize_shape[1] + resolution) // 2, :resolution]
        elif location == "right":
            new_img = img[(resize_shape[1] - resolution) // 2:(resize_shape[1] + resolution) // 2, -resolution:]
        else:
            new_img = img[(resize_shape[1] - resolution) // 2:(resize_shape[1] + resolution) // 2,
                      (resize_shape[0] - resolution) // 2:(resize_shape[0] + resolution) // 2]
        if brightness_norm:
            obj_v = np.mean(new_img)
            value = brightness_mean - obj_v
            new_img = cv2.add(new_img, value)
        imgs.append(new_img)
    return imgs

## Target images
In the first step, we get the target images.

In [10]:
# IMG_DIR = "/home/xavier/Documents/dataset/Welch/Zethus"
IMG_DIR = "/home/xavier/Documents/dataset/Welch/trainingset2/trainingset2"
img_pattern = re.compile(r'.*_(\d{4}).jpg')

img_names = []
img_names_rep = []
for strain_dir in glob.glob(os.path.join(IMG_DIR, "*")):
    if len(glob.glob(os.path.join(strain_dir, "Scope*"))) > 1:
        scope1, scope2 = random.sample(glob.glob(os.path.join(strain_dir, "Scope*")), 2)
        img_files = sorted(glob.glob(os.path.join(scope1, "*_????.jpg")))
        if img_files:
            last_img_file = img_files[-1]
            img_names.append(last_img_file)
            mid_img_file = img_files[600]
            img_names.append(mid_img_file)
        img_files = sorted(glob.glob(os.path.join(scope2, "*_????.jpg")))
        if img_files:
            last_img_file = img_files[-1]
            img_names_rep.append(last_img_file)
            mid_img_file = img_files[600]
            img_names_rep.append(mid_img_file)
    # for scope_dir in glob.glob(os.path.join(strain_dir, "Scope*")):
    #     img_files = sorted(glob.glob(os.path.join(scope_dir, "*_????.jpg")))
    #     if img_files:
    #         last_img_file = img_files[-1]
    #         imgs.append(last_img_file)
    # match = img_pattern.search(last_img_file)
    # if match:
    #     digits = match.group(1)

# Check the running loss

In [11]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

model_dict = {
    5: "/home/xavier/PycharmProjects/training-runs/new/e5/00001-stylegan2-trainingset2-gpus4-batch96-gamma10",
    7: "/home/xavier/PycharmProjects/training-runs/new/e7/00001-stylegan2-trainingset2-gpus4-batch96-gamma10",
    10: "/home/xavier/PycharmProjects/training-runs/new/e10/00001-stylegan2-trainingset2-gpus4-batch96-gamma10",
    12: "/home/xavier/PycharmProjects/training-runs/new/e12/00008-stylegan2-trainingset2-gpus4-batch96-gamma10",
    13: "/home/xavier/PycharmProjects/training-runs/new/e13/00008-stylegan2-trainingset2-gpus4-batch96-gamma10",
    14: "/home/xavier/PycharmProjects/training-runs/new/e14/00010-stylegan2-trainingset2-gpus4-batch96-gamma10",
    18: "/home/xavier/PycharmProjects/training-runs/new/e18/00001-stylegan2-trainingset2-gpus4-batch96-gamma10",
}
for e_dim in model_dict:
    log_path = model_dict[e_dim]

    # Initialize an event accumulator
    event_acc = EventAccumulator(log_path, size_guidance={'scalars': 0})
    event_acc.Reload()  # Load the log files

    # List all scalar tags
    all_tags = event_acc.Tags()['scalars']

    # Filter tags that start with "Loss/"
    loss_tags = [tag for tag in all_tags if tag.startswith('Loss/G') or tag.startswith('Loss/D')]

    loss_tot = 0
    # Extract and print the loss values at epoch 1411 for these tags
    for tag in loss_tags:
        loss_values = event_acc.Scalars(tag)
        for event in loss_values:
            if event.step == 1411:
                loss_tot += event.value
                print(f"{tag} at epoch {event.step}: {event.value}")
    print(f"e = {e_dim}, loss = {loss_tot}")

# Execute Model and Recalculate Loss

In [12]:
os.environ['CC'] = "/usr/bin/gcc-9"
os.environ['CXX'] = "/usr/bin/g++-9"
device = torch.device('cuda')
model_dict = {
    5: "/home/xavier/PycharmProjects/training-runs/new/e5/00001-stylegan2-trainingset2-gpus4-batch96-gamma10/network-snapshot-001461.pkl",
    7: "/home/xavier/PycharmProjects/training-runs/new/e7/00001-stylegan2-trainingset2-gpus4-batch96-gamma10/network-snapshot-001461.pkl",
    10: "/home/xavier/PycharmProjects/training-runs/new/e10/00001-stylegan2-trainingset2-gpus4-batch96-gamma10/network-snapshot-001411.pkl",
    12: "/home/xavier/PycharmProjects/training-runs/new/e12/00008-stylegan2-trainingset2-gpus4-batch96-gamma10/network-snapshot-001461.pkl",
    13: "/home/xavier/PycharmProjects/training-runs/new/e13/00008-stylegan2-trainingset2-gpus4-batch96-gamma10/network-snapshot-001461.pkl",
    14: "/home/xavier/PycharmProjects/training-runs/new/e14/00010-stylegan2-trainingset2-gpus4-batch96-gamma10/network-snapshot-001461.pkl",
    18: "/home/xavier/PycharmProjects/training-runs/new/e18/00001-stylegan2-trainingset2-gpus4-batch96-gamma10/network-snapshot-001461.pkl",
}
OUT_DIR = "/home/xavier/PycharmProjects/TuringTest/images"
os.makedirs(OUT_DIR, exist_ok=True)


def save_image(image, subdir, filename):
    path = os.path.join(OUT_DIR, subdir)
    if not os.path.exists(path):
        os.makedirs(path)
    cv2.imwrite(os.path.join(path, filename), image)


imgs_orig = []
imgs_other = []
imgs_rep = []
for i, img in enumerate(img_names):
    left_crop, right_crop = resize_crop(img)
    save_image(left_crop, 'left_crops', os.path.basename(img))
    save_image(right_crop, 'right_crops', os.path.basename(img))
    imgs_orig.append(left_crop[np.newaxis, np.newaxis, :, :])
    imgs_other.append(right_crop[np.newaxis, np.newaxis, :, :])
for img, couple_name in zip(img_names_rep, img_names):
    center_crop = resize_crop(img, locations=['center'])[0]
    save_image(center_crop, 'couple_crops', os.path.basename(couple_name))
    imgs_rep.append(center_crop[np.newaxis, np.newaxis, :, :])

# Calculate cosine similarity between inception-V3 features

In [13]:

detector_url = 'https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/metrics/inception-2015-12-05.pkl'
detector_kwargs = dict(return_features=True)  # Return raw features before the softmax layer.
detector = metric_utils.get_feature_detector(detector_url, device=torch.device('cuda'))

# Preprocess and calculate features for original and other images
img_l_features, img_r_features, img_rep_features = [], [], []

for img_l, img_r, img_rep in zip(imgs_orig, imgs_other, imgs_rep):
    img_l_tensor = torch.tensor(img_l, device=device, dtype=torch.float32).repeat([1, 3, 1, 1])
    img_r_tensor = torch.tensor(img_r, device=device, dtype=torch.float32).repeat([1, 3, 1, 1])
    img_rep_tensor = torch.tensor(img_rep, device=device, dtype=torch.float32).repeat([1, 3, 1, 1])
    img_l_features.append(detector(img_l_tensor))
    img_r_features.append(detector(img_r_tensor))
    img_rep_features.append(detector(img_rep_tensor))

# Initialize an empty DataFrame
similarities_table = pd.DataFrame()

# Calculate similarities between left and right images (original and other)
similarities_lr = []
similarities_lrep = []
for feat_l, feat_r, feat_rep in zip(img_l_features, img_r_features, img_rep_features):
    sim1 = cosine_similarity(feat_l.flatten(start_dim=1), feat_r.flatten(start_dim=1)).item()
    similarities_lr.append(sim1)
    sim2 = cosine_similarity(feat_l.flatten(start_dim=1), feat_rep.flatten(start_dim=1)).item()
    similarities_lrep.append(sim2)
similarities_table['Left-Right'] = similarities_lr
similarities_table['Left-Rep'] = similarities_lrep

# Execute models and calculate loss

In [14]:
from training.new_loss import StyleGAN2Loss

loss_all = {}
# Now calculate and add the similarities for each model's reconstructions
for model_e, model_path in model_dict.items():
    with dnnlib.util.open_url(model_path) as fp:
        models = legacy.load_network_pkl(fp)
    E, G, D = models['E_ema'].to(device), models['G_ema'].to(device), models['D'].to(device)

    loss = StyleGAN2Loss(device, G, D, E)
    loss_all[model_e] = 0
    similarities = []
    for feat_l, img_l, img_r, img_name in zip(img_l_features, imgs_orig, imgs_other, img_names):
        img_l_tensor = torch.tensor(img_l, device=device, dtype=torch.float32).div(127.5).sub(1)
        img_r_tensor = torch.tensor(img_r, device=device, dtype=torch.float32).div(127.5).sub(1)
        mu, logvar = E.mu_var(img_l_tensor, None)
        recon = G(mu, None).detach()
        recon_clipped = torch.clip(recon, -1, 1)
        recon_rescaled = recon_clipped.add(1).div(2).mul(255).type(torch.uint8)
        recon_output = recon_rescaled.detach().cpu().numpy()[0, 0]
        subdir = f'dim_{model_e}_reconstructions'
        save_image(recon_output, subdir, os.path.basename(img_name))
        recon_rescaled = recon_rescaled.repeat([1, 3, 1, 1])
        feat_r = detector(recon_rescaled)

        sim = cosine_similarity(feat_l.flatten(start_dim=1), feat_r.flatten(start_dim=1)).item()
        similarities.append(sim)

        # Calculate loss
        # L_tot = L_G + L_D + L_VAE + L_reg
        # L_G = loss_Gmain + loss_Gconstraints + loss_Gpl
        gen_logits = loss.run_D([img_l_tensor, recon], [None, None], blur_sigma=0)
        loss_Gmain = torch.nn.functional.softplus(-gen_logits)
        # L_D = loss_Dreal + loss_Dgen + gradient_penalty
        real_logits = loss.run_D([img_r_tensor, img_l_tensor], [None, None], blur_sigma=0)
        loss_Dreal = torch.nn.functional.softplus(-real_logits)
        loss_Dgen = torch.nn.functional.softplus(gen_logits)
        # L_VAE = KLD_loss
        # Lreg = loss_Dr1
        loss_all[model_e] += (loss_Gmain + loss_Dreal + loss_Dgen).cpu().numpy()[0][0]
    similarities_table[f'dim={model_e}'] = similarities
    print(f"{model_e}, {loss_all[model_e]}")
similarities_table.to_csv(os.path.join(OUT_DIR, "similarities_table.csv"))
similarities_table

# Recover from error

In [26]:
# import shutil
# 
# IMG_DIR = "/home/xavier/Documents/dataset/Welch/trainingset2/trainingset2"
# SOURCE_DIR = "/media/xavier/SHGP31/dataset/Welch/trainingset2/trainingset2"
# img_pattern = re.compile(r'.*_(\d{4}).jpg')
# 
# for strain_dir in glob.glob(os.path.join(IMG_DIR, "*")):
#     strain_base = os.path.basename(strain_dir)
#     for scope in glob.glob(os.path.join(strain_dir, "Scope*")):
#         scope_base = os.path.basename(scope)
#         img_files = sorted(glob.glob(os.path.join(scope, "*_????.jpg")))
#         if img_files:
#             last_img_file = img_files[-1]
#             print(last_img_file)
#             img_base = os.path.basename(last_img_file)
#             shutil.copy(os.path.join(SOURCE_DIR, strain_base, scope_base, img_base),
#                         os.path.join(IMG_DIR, strain_base, scope_base, img_base))

# Evaluation Metrics

In [5]:
# similarities_table.to_csv(os.path.join(OUT_DIR, "similarities_table.csv"))
similarities_table = pd.read_csv(os.path.join(OUT_DIR, "similarities_table.csv"))
plt.figure()
plt.hist(similarities_table['Left-Right'], bins=20, alpha=0.5, label=f'Left-Right')
plt.hist(similarities_table['Left-Rep'], bins=20, alpha=0.5, label=f'Left-Rep')
# Plotting histograms for each model
for model_e in model_dict.keys():
    plt.hist(similarities_table[f'dim={model_e}'], bins=20, alpha=0.5, label=f'dim={model_e}')
plt.xlabel('Cosine Similarity')
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [8]:
similarities_table.drop(columns=["Unnamed: 0"], inplace=True)
# Corrected loading and column naming
similarities_table.columns = ['Left-Right', 'Left-Rep', 'dim=7', 'dim=10', 'dim=12', 'dim=13', 'dim=14', 'dim=18']

# Perform ANOVA across the models
model_columns = ['Left-Rep', 'dim=7', 'dim=10', 'dim=12', 'dim=13', 'dim=14', 'dim=18']
anova_result = stats.f_oneway(*(similarities_table[col] for col in model_columns))

print(f"ANOVA result across models: F={anova_result.statistic:.4f}, p={anova_result.pvalue:.4f}")

# Check if there's a significant difference
if anova_result.pvalue < 0.05:
    print("There is a statistically significant difference between the models.")
else:
    print("There is no statistically significant difference between the models.")

# Preparing data for post-hoc Tukey's HSD if needed
if anova_result.pvalue < 0.05:
    data = np.concatenate([similarities_table[col].values for col in model_columns])
    labels = np.concatenate([[col] * len(similarities_table) for col in model_columns])

    # Performing Tukey's HSD
    tukey_result = pairwise_tukeyhsd(data, labels, alpha=0.05)
    print(tukey_result)

    # Identifying the best model based on mean similarity scores
    mean_similarities = similarities_table[model_columns].mean().sort_values(ascending=False)
    best_model = mean_similarities.idxmax()
    best_mean_similarity = mean_similarities.max()

    print(
        f"\nThe best model based on mean cosine similarity is {best_model} with a score of {best_mean_similarity:.4f}.")

    # Comparing "Left-Right" to the best model using a t-test
    t_stat, p_value = stats.ttest_ind(similarities_table['Left-Right'], similarities_table[best_model])
    print(f"\nT-test comparing 'Left-Right' to '{best_model}': t={t_stat:.4f}, p={p_value:.4f}")
    if p_value < 0.05:
        print("There is a statistically significant difference between 'Left-Right' and the best model.")
    else:
        print("There is no statistically significant difference between 'Left-Right' and the best model.")
else:
    print("\nSince there's no significant difference between models, we skip further analysis.")

t_stat, p_value = stats.ttest_ind(similarities_table['Left-Right'], similarities_table['dim=13'])
print(f"\nT-test comparing 'Left-Right' to 'dim=13': t={t_stat:.4f}, p={p_value:.4f}")
if p_value < 0.05:
    print("There is a statistically significant difference between 'Left-Right' and the best model.")
else:
    print("There is no statistically significant difference between 'Left-Right' and the best model.")